## Homegate scraping with selenium

Hopefully this works better and doesn't have inconsistencies.

In [13]:
# Preliminary
import pandas as pd
import requests
import re
import time
from bs4 import BeautifulSoup
import os
from selenium import webdriver
import time
from selenium.webdriver.common.by import By


os.chdir('/Users/drutna/Documents/pp4rs/assignment/pp4r_final_assignments') # change this to your path

In [15]:
chromedriver_name = 'chromedriver'
browser = webdriver.Chrome()

Trying with just one site first:

In [3]:
browser.get('https://www.homegate.ch/rent/3003551480')
time.sleep(10)
data = [dd.text for dd in browser.find_elements(By.TAG_NAME, 'dd')]
categories = [dt.text for dt in browser.find_elements(By.TAG_NAME, 'dt')]
df = pd.DataFrame([dict(zip(categories, data))])
address = browser.find_element(By.CLASS_NAME, 'AddressDetails_addressDetails_wuB1A .AddressDetails_address_i3koO').text
df['Address'] = address
df

,Net rent:,Add'l expenses:,Rent:,Available from:,Type:,No. of rooms:,Floor:,Surface living:,Listing ID,Object ref.,Address
0,"CHF 3,500.–",CHF 450.–,"CHF 3,950.–",01.11.2023,Apartment,3.5,1,114 m2,3003551480,iaao5.0hfm2,"Käferholzstrasse 32, 8057 Zürich"


This creates a nice dataframe with various data on the listing.

Trying with another test site:

In [4]:
browser = webdriver.Chrome()
browser.get('https://www.homegate.ch/rent/3003556313')
time.sleep(10)
data = [dd.text for dd in browser.find_elements(By.TAG_NAME, 'dd')]
categories = [dt.text for dt in browser.find_elements(By.TAG_NAME, 'dt')]
df = pd.DataFrame([dict(zip(categories, data))])
address = browser.find_element(By.CLASS_NAME, 'AddressDetails_addressDetails_wuB1A .AddressDetails_address_i3koO').text
df['Address'] = address
df

,Net rent:,Add'l expenses:,Rent:,Available from:,Type:,No. of rooms:,Floor:,Number of floors:,Year built:,Listing ID,Object ref.,Address
0,"CHF 1,950.–",CHF 300.–,"CHF 2,250.–",By agreement,Roof flat,3.5,2,2,1650,3003556313,ksw9m.b00oe,"Unterdorfstrasse 15, 8105 Watt"


Seems to work. Now with multiple listings...

Go through all 1-50 of the search pages and find all the links in each page:

In [16]:
urls = []
for i in range(1, 50):    
    search_url_pattern = 'https://www.homegate.ch/rent/real-estate/city-zurich/matching-list?ep={}&be=2000&o=dateCreated-desc'
    search_url = search_url_pattern.format(i)
    response = browser.get(search_url)
    elems = browser.find_elements(By.TAG_NAME, "a")
    for elem in elems:
        urls.append(elem.get_attribute("href"))

Narrow the links down to only those that are of rental property listings:

In [17]:
pattern = 'https://www.homegate.ch/rent/\d'
urls2 = [s for s in urls if re.match(pattern, s)]
urls2

['https://www.homegate.ch/rent/4000128038',
 'https://www.homegate.ch/rent/3003561923',
 'https://www.homegate.ch/rent/3003561870',
 'https://www.homegate.ch/rent/3003561634',
 'https://www.homegate.ch/rent/3003561592',
 'https://www.homegate.ch/rent/3003561552',
 'https://www.homegate.ch/rent/3003561511',
 'https://www.homegate.ch/rent/4000127430',
 'https://www.homegate.ch/rent/4000127429',
 'https://www.homegate.ch/rent/4000127428',
 'https://www.homegate.ch/rent/4000127427',
 'https://www.homegate.ch/rent/4000127426',
 'https://www.homegate.ch/rent/4000127425',
 'https://www.homegate.ch/rent/4000127423',
 'https://www.homegate.ch/rent/3003561460',
 'https://www.homegate.ch/rent/4000127315',
 'https://www.homegate.ch/rent/3003561370',
 'https://www.homegate.ch/rent/3003561365',
 'https://www.homegate.ch/rent/3003561363',
 'https://www.homegate.ch/rent/3003561067',
 'https://www.homegate.ch/rent/3003560962',
 'https://www.homegate.ch/rent/3003560957',
 'https://www.homegate.ch/rent/3

In [20]:
len(urls2)

980

Save the list of urls:

In [25]:
df = pd.DataFrame(data={"urls": urls2})
df.to_csv("./file.csv", sep=',',index=False)

We have 980 urls which we should check with selenium. In the following we scrape the multiple sites. This took about 1.5 hours.

In [27]:
df2 = pd.DataFrame()
counter = 0
for hg_url in urls2:
    try:
        response = browser.get(hg_url)
        time.sleep(5)
        data = [dd.text for dd in browser.find_elements(By.TAG_NAME, 'dd')]
        categories = [dt.text for dt in browser.find_elements(By.TAG_NAME, 'dt')]
        df = pd.DataFrame([dict(zip(categories, data))])
        df['Address'] = browser.find_element(By.CLASS_NAME, 'AddressDetails_addressDetails_wuB1A .AddressDetails_address_i3koO').text
        df['URL'] = hg_url
        df2 = pd.concat([df2, df], ignore_index = True)   
    except: 
        pass
    counter += 1
    if counter%10 == 0:
        print(counter)
        df2.to_csv('Homegate_data.csv', index = False)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
